In [3]:
! pip install rouge_score evaluate

  Using cached rouge_score-0.1.2-py3-none-any.whl
  Using cached evaluate-0.4.2-py3-none-any.whl.metadata (9.3 kB)
  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
Using cached evaluate-0.4.2-py3-none-any.whl (84 kB)
Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.5.0-py3-none-any.whl (316 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
Using cached xxhash-3.4.1-cp310-cp310

In [4]:
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import os
import torch

from datasets import Dataset, load_metric

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Baselines'

'baseline_flan_t5_base_adj_shortdialogue_predictions (1).gsheet'
 baseline_flan_t5_base_adj_shortdialogue_predictions.csv
 baseline_flan_t5_base_adj_shortdialogue_predictions.gsheet
 baseline_flan_t5_base_shortdialogue_predictions.csv
 baseline_flan_t5_base_shortdialogue_predictions.xlsx
 baseline_flan_t5_large_shortdialogue_predictions.csv
 baseline_flan_t5_large_shortdialogue_predictions.xlsx
 led_lora_short_predictions.csv
 led_lora_short_predictions.xlsx


In [6]:
!ls '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Bio'

flan_t5_base_adj_NER_bio_predictions.csv
flan_t5_base_adj_NER_bio_predictions.gsheet
flan_t5_base_NER_bio_predictions.csv
flan_t5_base_NER_bio_predictions.xlsx
flan_t5_large_shortdialogue_NER_bio_predictions.csv
flan_t5_large_shortdialogue_NER_bio_predictions.xlsx
led_lora_bio_short_unique_predictions.csv
led_lora_bio_short_unique_predictions.xlsx
Old
original_results


In [7]:
!ls '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Clinical'

 flan_t5_base_NER_clinical_adj_predictions.csv
 flan_t5_base_NER_clinical_adj_predictions.xlsx
 flan_t5_base_NER_clinical_predictions.csv
 flan_t5_base_NER_clinical_predictions.xlsx
 flan_t5_large_shortdialogue_NER_clinical_predictions.csv
 flan_t5_large_shortdialogue_NER_clinical_predictions.xlsx
 led_lora_clinical_short_unique_predictions.csv
 led_lora_clinical_short_unique_predictions.xlsx
'original results'


Load data

In [8]:
!ls '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Keybert'

flan_t5_base_adj_NER_keybert_predictions.csv
flan_t5_base_adj_NER_keybert_predictions.gsheet
flan_t5_base_NER_keybert_predictions.csv
flan_t5_base_NER_keybert_predictions.xlsx
flan_t5_large_shortdialogue_keybert_predictions.csv
flan_t5_large_shortdialogue_keybert_predictions.xlsx
led_lora_keybert_short_unique_predictions.csv
led_lora_keybert_short_unique_predictions.xlsx
original_results


In [9]:
# Path to the files
flan_t5_base = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Baselines/baseline_flan_t5_base_shortdialogue_predictions.csv'
flan_t5_base_Bio = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Bio/flan_t5_base_NER_bio_predictions.csv'
flan_t5_base_Clinical = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Clinical/flan_t5_base_NER_clinical_predictions.csv'
flan_t5_base_keybert = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Keybert/flan_t5_base_NER_keybert_predictions.csv'

flan_t5_base_adj = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Baselines/baseline_flan_t5_base_adj_shortdialogue_predictions.csv'
flan_t5_base_Bio_adj = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Bio/flan_t5_base_adj_NER_bio_predictions.csv'
flan_t5_base_Clinical_adj = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Clinical/flan_t5_base_NER_clinical_adj_predictions.csv'
flan_t5_base_keybert_adj = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Keybert/flan_t5_base_adj_NER_keybert_predictions.csv'

flan_t5_large = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Baselines/baseline_flan_t5_large_shortdialogue_predictions.csv'
flan_t5_large_Bio = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Bio/flan_t5_large_shortdialogue_NER_bio_predictions.csv'
flan_t5_large_Clinical = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Clinical/flan_t5_large_shortdialogue_NER_clinical_predictions.csv'
flan_t5_large_keybert = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Keybert/flan_t5_large_shortdialogue_keybert_predictions.csv'

LED_large_shortdialogue = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Baselines/led_lora_short_predictions.csv'
LED_large_Bio_short_dialogues = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Bio/led_lora_bio_short_unique_predictions.csv'
LED_large_Clinical_shortdialogue = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Clinical/led_lora_clinical_short_unique_predictions.csv'
LED_large_keybert_shortdialogues = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Keybert/led_lora_keybert_short_unique_predictions.csv'


In [12]:
# Load the datasets
flan_t5_base_df = pd.read_csv(flan_t5_base)
flan_t5_base_Bio_df = pd.read_csv(flan_t5_base_Bio)
flan_t5_base_Clinical_df = pd.read_csv(flan_t5_base_Clinical)
flan_t5_base_keybert_df = pd.read_csv(flan_t5_base_keybert)

flan_t5_base_adj_df = pd.read_csv(flan_t5_base_adj)
flan_t5_base_Bio_adj_df = pd.read_csv(flan_t5_base_Bio_adj)
flan_t5_base_Clinical_adj_df = pd.read_csv(flan_t5_base_Clinical_adj)
flan_t5_base_keybert_adj_df = pd.read_csv(flan_t5_base_keybert_adj)

flan_t5_large_df = pd.read_csv(flan_t5_large)
flan_t5_large_Bio_df = pd.read_csv(flan_t5_large_Bio)
flan_t5_large_Clinical_df = pd.read_csv(flan_t5_large_Clinical)
flan_t5_large_keybert_df = pd.read_csv(flan_t5_large_keybert)

LED_large_shortdialogue_df = pd.read_csv(LED_large_shortdialogue)
LED_large_Bio_short_dialogues_df = pd.read_csv(LED_large_Bio_short_dialogues)
LED_large_Clinical_shortdialogue_df = pd.read_csv(LED_large_Clinical_shortdialogue)
LED_large_keybert_shortdialogues_df = pd.read_csv(LED_large_keybert_shortdialogues)


In [13]:
# Check shapes
print("Shape of flan_t5_base:")
print(flan_t5_base_df.shape)

print("Shape of flan_t5_base_Bio:")
print(flan_t5_base_Bio_df.shape)

print("Shape of flan_t5_base_Clinical:")
print(flan_t5_base_Clinical_df.shape)

print("Shape of flan_t5_base_keybert:")
print(flan_t5_base_keybert_df.shape)

print("\n\n")

print("Shape of flan_t5_base_adj:")
print(flan_t5_base_adj_df.shape)

print("Shape of flan_t5_base_Bio:")
print(flan_t5_base_Bio_adj_df.shape)

print("Shape of flan_t5_base_Clinical:")
print(flan_t5_base_Clinical_adj_df.shape)

print("Shape of flan_t5_base_keybert:")
print(flan_t5_base_keybert_adj_df.shape)

print("\n\n")

print("Shape of flan_t5_large:")
print(flan_t5_large_df.shape)

print("Shape of flan_t5_large_Bio:")
print(flan_t5_large_Bio_df.shape)

print("Shape of flan_t5_large_Clinical:")
print(flan_t5_large_Clinical_df.shape)

print("Shape of flan_t5_large_keybert:")
print(flan_t5_large_keybert_df.shape)

print("\n\n")

print("Shape of LED_large_shortdialogue:")
print(LED_large_shortdialogue_df.shape)

print("Shape of LED_large_Bio_short_dialogues:")
print(LED_large_Bio_short_dialogues_df.shape)

print("Shape of LED_large_Clinical_shortdialogue:")
print(LED_large_Clinical_shortdialogue_df.shape)

print("Shape of LED_large_keybert_shortdialogues:")
print(LED_large_keybert_shortdialogues_df.shape)


Shape of flan_t5_base:
(222, 2)
Shape of flan_t5_base_Bio:
(222, 3)
Shape of flan_t5_base_Clinical:
(222, 3)
Shape of flan_t5_base_keybert:
(222, 3)



Shape of flan_t5_base_adj:
(222, 3)
Shape of flan_t5_base_Bio:
(222, 3)
Shape of flan_t5_base_Clinical:
(222, 3)
Shape of flan_t5_base_keybert:
(222, 3)



Shape of flan_t5_large:
(222, 3)
Shape of flan_t5_large_Bio:
(222, 3)
Shape of flan_t5_large_Clinical:
(222, 3)
Shape of flan_t5_large_keybert:
(222, 3)



Shape of LED_large_shortdialogue:
(222, 3)
Shape of LED_large_Bio_short_dialogues:
(222, 3)
Shape of LED_large_Clinical_shortdialogue:
(222, 3)
Shape of LED_large_keybert_shortdialogues:
(222, 3)


Display for two row for flan_t5_base

In [14]:
# Display the first two rows of each DataFrame
print("First 2 rows of flan_t5_base:")
print(flan_t5_base_df.head(2))

print("First 2 rows of flan_t5_base_Bio:")
print(flan_t5_base_Bio_df.head(2))

print("First 2 rows of flan_t5_base_Clinical:")
print(flan_t5_base_Clinical_df.head(2))

print("First 2 rows of flan_t5_base_keybert:")
print(flan_t5_base_keybert_df.head(2))

First 2 rows of flan_t5_base:
                                           Reference  \
0  The patient lives with her husband of 48 years...   
1  ONCOLOGIC: No history of any cancer, change in...   

                                          Prediction  
0  The patient is married. She lives in the same ...  
1  No change in moles, no change in moles, no sud...  
First 2 rows of flan_t5_base_Bio:
                                            Dialogue  \
0  Doctor: Hi, there. Patient: Hi. Guest_family: ...   
1  Doctor: Were you ever diagnosed with any kind ...   

                                           Reference  \
0  The patient lives with her husband of 48 years...   
1  ONCOLOGIC: No history of any cancer, change in...   

                                          Prediction  
0  The patient is a 40-year-old male who lives in...  
1  No cancer. No rashes, changes in moles, or wei...  
First 2 rows of flan_t5_base_Clinical:
                                           Dialogies  \
0  D

Display for two row for flan_t5_base adj (finetuned)

In [15]:
# Display the first two rows of each DataFrame
print("First 2 rows of flan_t5_base:")
print(flan_t5_base_adj_df.head(2))

print("First 2 rows of flan_t5_base_Bio:")
print(flan_t5_base_Bio_adj_df.head(2))

print("First 2 rows of flan_t5_base_Clinical:")
print(flan_t5_base_Clinical_adj_df.head(2))

print("First 2 rows of flan_t5_base_keybert:")
print(flan_t5_base_keybert_adj_df.head(2))

First 2 rows of flan_t5_base:
                                            Dialogue  \
0  Doctor: Hi, there. Patient: Hi. Guest_family: ...   
1  Doctor: Were you ever diagnosed with any kind ...   

                                           Reference  \
0  The patient lives with her husband of 48 years...   
1  ONCOLOGIC: No history of any cancer, change in...   

                                          Prediction  
0  The patient is married. She lives in the same ...  
1  The patient denies any cancer, rashes, changes...  
First 2 rows of flan_t5_base_Bio:
                                            Dialogue  \
0  Doctor: Hi, there. Patient: Hi. Guest_family: ...   
1  Doctor: Were you ever diagnosed with any kind ...   

                                           Reference  \
0  The patient lives with her husband of 48 years...   
1  ONCOLOGIC: No history of any cancer, change in...   

                                          Prediction  
0  The patient is married for forty-eigh

Display for two row for flan_t5_large

In [17]:
print("First 2 rows of flan_t5_large:")
print(flan_t5_large_df.head(2))

print("First 2 rows of flan_t5_large_Bio:")
print(flan_t5_large_Bio_df.head(2))

print("First 2 rows of flan_t5_large_Clinical:")
print(flan_t5_large_Clinical_df.head(2))

print("First 2 rows of flan_t5_large_keybert:")
print(flan_t5_large_keybert_df.head(2))


First 2 rows of flan_t5_large:
                                           Dialogues  \
0  Doctor: Hi, there. Patient: Hi. Guest_family: ...   
1  Doctor: Were you ever diagnosed with any kind ...   

                                           Reference  \
0  The patient lives with her husband of 48 years...   
1  ONCOLOGIC: No history of any cancer, change in...   

                                          Prediction  
0  The patient is married. She lives with her hus...  
1  Noncontributory. No history of cancer, rashes,...  
First 2 rows of flan_t5_large_Bio:
                                            Dialogue  \
0  Doctor: Hi, there. Patient: Hi. Guest_family: ...   
1  Doctor: Were you ever diagnosed with any kind ...   

                                           Reference  \
0  The patient lives with her husband of 48 years...   
1  ONCOLOGIC: No history of any cancer, change in...   

                                          Prediction  
0  The patient is a 48-year-old marrie

Display for two row for LED LORA large

In [18]:
print("First 2 rows of LED_large_shortdialogue:")
print(LED_large_shortdialogue_df.head(2))

print("First 2 rows of LED_large_Bio_short_dialogues:")
print(LED_large_Bio_short_dialogues_df.head(2))

print("First 2 rows of LED_large_Clinical_shortdialogue:")
print(LED_large_Clinical_shortdialogue_df.head(2))

print("First 2 rows of LED_large_keybert_shortdialogues:")
print(LED_large_keybert_shortdialogues_df.head(2))

First 2 rows of LED_large_shortdialogue:
                                            Dialogue  \
0  Doctor: Hi, there. Patient: Hi. Guest_family: ...   
1  Doctor: Were you ever diagnosed with any kind ...   

                                           Reference  \
0  The patient lives with her husband of 48 years...   
1  ONCOLOGIC: No history of any cancer, change in...   

                                          Prediction  
0  The husband and wife have been married for for...  
1  The patient has no history of cancer in the pa...  
First 2 rows of LED_large_Bio_short_dialogues:
                                            Dialogue  \
0  Doctor: Hi, there. Patient: Hi. Guest_family: ...   
1  Doctor: Were you ever diagnosed with any kind ...   

                                           Reference  \
0  The patient lives with her husband of 48 years...   
1  ONCOLOGIC: No history of any cancer, change in...   

                                          Prediction  
0  The husband a

Show dialogue and reference list for Flan_T5_base

In [19]:
reference_ft5_base = flan_t5_base_df["Reference"].tolist()

In [20]:
len(reference_ft5_base)

222

In [21]:
Flan_t5_base_baseline_predictions = flan_t5_base_df["Prediction"].tolist()

In [22]:
len(Flan_t5_base_baseline_predictions)

222

In [23]:
Flan_t5_base_Bio_predictions = flan_t5_base_Bio_df["Prediction"].tolist()

In [24]:
len(Flan_t5_base_Bio_predictions)

222

In [25]:
Flan_t5_base_Clinical_predictions = flan_t5_base_Clinical_df["Prediction"].tolist()

In [26]:
len(Flan_t5_base_Clinical_predictions)

222

In [27]:
Flan_T5_base_KeyBert_predictions = flan_t5_base_keybert_df["Prediction"].tolist()

In [28]:
len(Flan_T5_base_KeyBert_predictions)

222

Show dialogue and reference list for Flan_T5_base adj (finetuned)

In [29]:
reference_ft5_base_adj = flan_t5_base_adj_df["Reference"].tolist()

In [30]:
len(reference_ft5_base_adj)

222

In [31]:
Flan_t5_base_baseline_predictions_adj = flan_t5_base_adj_df["Prediction"].tolist()

In [32]:
len(Flan_t5_base_baseline_predictions_adj)

222

In [33]:
Flan_t5_base_Bio_predictions_adj = flan_t5_base_Bio_adj_df["Prediction"].tolist()

In [34]:
len(Flan_t5_base_Bio_predictions_adj)

222

In [35]:
Flan_t5_base_Clinical_predictions_adj = flan_t5_base_Clinical_adj_df["Prediction"].tolist()

In [36]:
len(Flan_t5_base_Clinical_predictions_adj)

222

In [37]:
Flan_T5_base_KeyBert_predictions_adj = flan_t5_base_keybert_adj_df["Prediction"].tolist()

In [38]:
len(Flan_T5_base_KeyBert_predictions_adj)

222

Show dialogue and reference list for Flan_T5_large

In [39]:
reference_ft5_large = flan_t5_large_df["Reference"].tolist()

In [40]:
len(reference_ft5_large)

222

In [41]:
Flan_t5_large_baseline_predictions = flan_t5_large_df["Prediction"].tolist()

In [ ]:
len(Flan_t5_large_baseline_predictions)

In [42]:
Flan_t5_large_Bio_predictions = flan_t5_large_Bio_df["Prediction"].tolist()

In [43]:
len(Flan_t5_large_Bio_predictions)

222

In [44]:
Flan_t5_large_Clinical_predictions = flan_t5_large_Clinical_df["Prediction"].tolist()

In [45]:
len(Flan_t5_large_Clinical_predictions)

222

In [46]:
Flan_T5_large_KeyBert_predictions = flan_t5_large_keybert_df["Prediction"].tolist()

In [47]:
len(Flan_T5_large_KeyBert_predictions)

222

Show dialogue and reference list for LED_LORA (large)

In [48]:
reference_LED_LORA_large_baseline_shortdialogue = LED_large_shortdialogue_df["Reference"].tolist()

In [49]:
len(reference_LED_LORA_large_baseline_shortdialogue)

222

In [50]:
LED_LORA_large_shortdialogue_predictions = LED_large_shortdialogue_df["Prediction"].tolist()

In [51]:
len(LED_LORA_large_shortdialogue_predictions)

222

In [52]:
LED_LORA_large_shortdialogue_Bio_predictions = LED_large_Bio_short_dialogues_df["Prediction"].tolist()

In [53]:
len(LED_LORA_large_shortdialogue_Bio_predictions)

222

In [54]:
LED_LORA_large_shortdialogue_Clinical_predictions = LED_large_Clinical_shortdialogue_df["Prediction"].tolist()

In [55]:
len(LED_LORA_large_shortdialogue_Clinical_predictions)

222

In [56]:
LED_LORA_large_shortdialogue_keybert_predictions = LED_large_keybert_shortdialogues_df["Prediction"].tolist()

In [57]:
len(LED_LORA_large_shortdialogue_keybert_predictions)

222

Rouge Scores - Flan_T5 base on short dialogues

In [58]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(Flan_t5_base_baseline_predictions, reference_ft5_base):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")


Average ROUGE-1 Recall: 0.5049
Average ROUGE-1 Precision: 0.2601
Average ROUGE-1 F1: 0.2896
Average ROUGE-2 Recall: 0.2385
Average ROUGE-2 Precision: 0.1016
Average ROUGE-2 F1: 0.1218
Average ROUGE-L Recall: 0.4325
Average ROUGE-L Precision: 0.2302
Average ROUGE-L F1: 0.2529


In [59]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(Flan_t5_base_Bio_predictions, reference_ft5_base):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")


Average ROUGE-1 Recall: 0.5159
Average ROUGE-1 Precision: 0.2605
Average ROUGE-1 F1: 0.2918
Average ROUGE-2 Recall: 0.2424
Average ROUGE-2 Precision: 0.1060
Average ROUGE-2 F1: 0.1251
Average ROUGE-L Recall: 0.4371
Average ROUGE-L Precision: 0.2290
Average ROUGE-L F1: 0.2527


In [60]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(Flan_t5_base_Clinical_predictions, reference_ft5_base):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")


Average ROUGE-1 Recall: 0.5085
Average ROUGE-1 Precision: 0.2467
Average ROUGE-1 F1: 0.2818
Average ROUGE-2 Recall: 0.2341
Average ROUGE-2 Precision: 0.1019
Average ROUGE-2 F1: 0.1200
Average ROUGE-L Recall: 0.4268
Average ROUGE-L Precision: 0.2161
Average ROUGE-L F1: 0.2423


In [61]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(Flan_T5_base_KeyBert_predictions, reference_ft5_base):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")


Average ROUGE-1 Recall: 0.5167
Average ROUGE-1 Precision: 0.2500
Average ROUGE-1 F1: 0.2826
Average ROUGE-2 Recall: 0.2497
Average ROUGE-2 Precision: 0.1031
Average ROUGE-2 F1: 0.1240
Average ROUGE-L Recall: 0.4426
Average ROUGE-L Precision: 0.2203
Average ROUGE-L F1: 0.2464


Display for two row for flan_t5_base adj (finetuned)

In [62]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(Flan_t5_base_baseline_predictions_adj, reference_ft5_base_adj):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")


Average ROUGE-1 Recall: 0.5387
Average ROUGE-1 Precision: 0.2580
Average ROUGE-1 F1: 0.2978
Average ROUGE-2 Recall: 0.2577
Average ROUGE-2 Precision: 0.0985
Average ROUGE-2 F1: 0.1220
Average ROUGE-L Recall: 0.4635
Average ROUGE-L Precision: 0.2314
Average ROUGE-L F1: 0.2625


In [63]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(Flan_t5_base_Bio_predictions_adj, reference_ft5_base_adj):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")

Average ROUGE-1 Recall: 0.5226
Average ROUGE-1 Precision: 0.2518
Average ROUGE-1 F1: 0.2855
Average ROUGE-2 Recall: 0.2438
Average ROUGE-2 Precision: 0.0999
Average ROUGE-2 F1: 0.1200
Average ROUGE-L Recall: 0.4444
Average ROUGE-L Precision: 0.2209
Average ROUGE-L F1: 0.2480


In [64]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(Flan_t5_base_Clinical_predictions_adj, reference_ft5_base_adj):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")


Average ROUGE-1 Recall: 0.5300
Average ROUGE-1 Precision: 0.2468
Average ROUGE-1 F1: 0.2890
Average ROUGE-2 Recall: 0.2460
Average ROUGE-2 Precision: 0.0990
Average ROUGE-2 F1: 0.1190
Average ROUGE-L Recall: 0.4479
Average ROUGE-L Precision: 0.2164
Average ROUGE-L F1: 0.2498


In [65]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(Flan_T5_base_KeyBert_predictions_adj, reference_ft5_base_adj):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")

Average ROUGE-1 Recall: 0.5313
Average ROUGE-1 Precision: 0.2460
Average ROUGE-1 F1: 0.2868
Average ROUGE-2 Recall: 0.2571
Average ROUGE-2 Precision: 0.1042
Average ROUGE-2 F1: 0.1272
Average ROUGE-L Recall: 0.4552
Average ROUGE-L Precision: 0.2162
Average ROUGE-L F1: 0.2497


Rouge Scores - Flan_T5 large on short dialogues

In [66]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(Flan_t5_large_baseline_predictions, reference_ft5_large):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")


Average ROUGE-1 Recall: 0.5602
Average ROUGE-1 Precision: 0.2993
Average ROUGE-1 F1: 0.3269
Average ROUGE-2 Recall: 0.2885
Average ROUGE-2 Precision: 0.1325
Average ROUGE-2 F1: 0.1554
Average ROUGE-L Recall: 0.4833
Average ROUGE-L Precision: 0.2639
Average ROUGE-L F1: 0.2861


In [67]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(Flan_t5_large_Bio_predictions, reference_ft5_large):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")


Average ROUGE-1 Recall: 0.5357
Average ROUGE-1 Precision: 0.2897
Average ROUGE-1 F1: 0.3092
Average ROUGE-2 Recall: 0.2649
Average ROUGE-2 Precision: 0.1232
Average ROUGE-2 F1: 0.1373
Average ROUGE-L Recall: 0.4541
Average ROUGE-L Precision: 0.2519
Average ROUGE-L F1: 0.2657


In [68]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(Flan_t5_large_Clinical_predictions, reference_ft5_large):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")


Average ROUGE-1 Recall: 0.5284
Average ROUGE-1 Precision: 0.2887
Average ROUGE-1 F1: 0.3056
Average ROUGE-2 Recall: 0.2641
Average ROUGE-2 Precision: 0.1229
Average ROUGE-2 F1: 0.1386
Average ROUGE-L Recall: 0.4509
Average ROUGE-L Precision: 0.2534
Average ROUGE-L F1: 0.2643


In [69]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(Flan_T5_large_KeyBert_predictions, reference_ft5_large):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")


Average ROUGE-1 Recall: 0.5448
Average ROUGE-1 Precision: 0.3020
Average ROUGE-1 F1: 0.3160
Average ROUGE-2 Recall: 0.2756
Average ROUGE-2 Precision: 0.1288
Average ROUGE-2 F1: 0.1435
Average ROUGE-L Recall: 0.4629
Average ROUGE-L Precision: 0.2627
Average ROUGE-L F1: 0.2709


Rouge Scores - LED LORA large on short dialogues

In [70]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(LED_LORA_large_shortdialogue_predictions, reference_LED_LORA_large_baseline_shortdialogue):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")


Average ROUGE-1 Recall: 0.1458
Average ROUGE-1 Precision: 0.6198
Average ROUGE-1 F1: 0.2101
Average ROUGE-2 Recall: 0.0590
Average ROUGE-2 Precision: 0.2479
Average ROUGE-2 F1: 0.0853
Average ROUGE-L Recall: 0.0914
Average ROUGE-L Precision: 0.4579
Average ROUGE-L F1: 0.1351


In [71]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(LED_LORA_large_shortdialogue_Bio_predictions, reference_LED_LORA_large_baseline_shortdialogue):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")


Average ROUGE-1 Recall: 0.1139
Average ROUGE-1 Precision: 0.6384
Average ROUGE-1 F1: 0.1760
Average ROUGE-2 Recall: 0.0441
Average ROUGE-2 Precision: 0.2500
Average ROUGE-2 F1: 0.0686
Average ROUGE-L Recall: 0.0695
Average ROUGE-L Precision: 0.4727
Average ROUGE-L F1: 0.1106


In [72]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(LED_LORA_large_shortdialogue_Clinical_predictions, reference_LED_LORA_large_baseline_shortdialogue):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")


Average ROUGE-1 Recall: 0.1116
Average ROUGE-1 Precision: 0.6251
Average ROUGE-1 F1: 0.1714
Average ROUGE-2 Recall: 0.0431
Average ROUGE-2 Precision: 0.2475
Average ROUGE-2 F1: 0.0665
Average ROUGE-L Recall: 0.0693
Average ROUGE-L Precision: 0.4649
Average ROUGE-L F1: 0.1095


In [73]:
from rouge_score import rouge_scorer

# Instantiate ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual scores
rouge1_recall = []
rouge1_precision = []
rouge1_f1 = []
rouge2_recall = []
rouge2_precision = []
rouge2_f1 = []
rougeL_recall = []
rougeL_precision = []
rougeL_f1 = []

# Compute ROUGE scores for each pair of prediction and reference
for pred, ref in zip(LED_LORA_large_shortdialogue_keybert_predictions, reference_LED_LORA_large_baseline_shortdialogue):
    score = scorer.score(pred, ref)
    rouge1_recall.append(score['rouge1'].recall)
    rouge1_precision.append(score['rouge1'].precision)
    rouge1_f1.append(score['rouge1'].fmeasure)
    rouge2_recall.append(score['rouge2'].recall)
    rouge2_precision.append(score['rouge2'].precision)
    rouge2_f1.append(score['rouge2'].fmeasure)
    rougeL_recall.append(score['rougeL'].recall)
    rougeL_precision.append(score['rougeL'].precision)
    rougeL_f1.append(score['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1_recall_baseline = sum(rouge1_recall) / len(rouge1_recall)
avg_rouge1_precision_baseline = sum(rouge1_precision) / len(rouge1_precision)
avg_rouge1_f1_baseline = sum(rouge1_f1) / len(rouge1_f1)
avg_rouge2_recall_baseline = sum(rouge2_recall) / len(rouge2_recall)
avg_rouge2_precision_baseline = sum(rouge2_precision) / len(rouge2_precision)
avg_rouge2_f1_baseline = sum(rouge2_f1) / len(rouge2_f1)
avg_rougeL_recall_baseline = sum(rougeL_recall) / len(rougeL_recall)
avg_rougeL_precision_baseline = sum(rougeL_precision) / len(rougeL_precision)
avg_rougeL_f1_baseline = sum(rougeL_f1) / len(rougeL_f1)

# Print average ROUGE scores
print(f"Average ROUGE-1 Recall: {avg_rouge1_recall_baseline:.4f}")
print(f"Average ROUGE-1 Precision: {avg_rouge1_precision_baseline:.4f}")
print(f"Average ROUGE-1 F1: {avg_rouge1_f1_baseline:.4f}")
print(f"Average ROUGE-2 Recall: {avg_rouge2_recall_baseline:.4f}")
print(f"Average ROUGE-2 Precision: {avg_rouge2_precision_baseline:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2_f1_baseline:.4f}")
print(f"Average ROUGE-L Recall: {avg_rougeL_recall_baseline:.4f}")
print(f"Average ROUGE-L Precision: {avg_rougeL_precision_baseline:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL_f1_baseline:.4f}")


Average ROUGE-1 Recall: 0.1141
Average ROUGE-1 Precision: 0.6180
Average ROUGE-1 F1: 0.1732
Average ROUGE-2 Recall: 0.0454
Average ROUGE-2 Precision: 0.2474
Average ROUGE-2 F1: 0.0689
Average ROUGE-L Recall: 0.0703
Average ROUGE-L Precision: 0.4600
Average ROUGE-L F1: 0.1099


Dataframe to show all metrics

In [76]:
from rouge_score import rouge_scorer
import pandas as pd

# Define a function to calculate ROUGE scores
def calculate_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1_recall = []
    rouge1_precision = []
    rouge1_f1 = []
    rouge2_recall = []
    rouge2_precision = []
    rouge2_f1 = []
    rougeL_recall = []
    rougeL_precision = []
    rougeL_f1 = []

    for pred, ref in zip(predictions, references):
        score = scorer.score(pred, ref)
        rouge1_recall.append(score['rouge1'].recall)
        rouge1_precision.append(score['rouge1'].precision)
        rouge1_f1.append(score['rouge1'].fmeasure)
        rouge2_recall.append(score['rouge2'].recall)
        rouge2_precision.append(score['rouge2'].precision)
        rouge2_f1.append(score['rouge2'].fmeasure)
        rougeL_recall.append(score['rougeL'].recall)
        rougeL_precision.append(score['rougeL'].precision)
        rougeL_f1.append(score['rougeL'].fmeasure)

    avg_scores = {
        'rouge1_recall': sum(rouge1_recall) / len(rouge1_recall),
        'rouge1_precision': sum(rouge1_precision) / len(rouge1_precision),
        'rouge1_f1': sum(rouge1_f1) / len(rouge1_f1),
        'rouge2_recall': sum(rouge2_recall) / len(rouge2_recall),
        'rouge2_precision': sum(rouge2_precision) / len(rouge2_precision),
        'rouge2_f1': sum(rouge2_f1) / len(rouge2_f1),
        'rougeL_recall': sum(rougeL_recall) / len(rougeL_recall),
        'rougeL_precision': sum(rougeL_precision) / len(rougeL_precision),
        'rougeL_f1': sum(rougeL_f1) / len(rougeL_f1),
    }
    return avg_scores

# Define your predictions and references for each model
models_predictions_references = {
    'flan_t5_base': (Flan_t5_base_baseline_predictions, reference_ft5_base),
    'flan_t5_base_Bio': (Flan_t5_base_Bio_predictions, reference_ft5_base),
    'flan_t5_base_Clinical': (Flan_t5_base_Clinical_predictions, reference_ft5_base),
    'flan_t5_base_KeyBert': (Flan_T5_base_KeyBert_predictions, reference_ft5_base),
    'flan_t5_base_adj': (Flan_t5_base_baseline_predictions_adj, reference_ft5_base_adj),
    'flan_t5_base_Bio_adj': (Flan_t5_base_Bio_predictions_adj, reference_ft5_base_adj),
    'flan_t5_base_Clinical_adj': (Flan_t5_base_Clinical_predictions_adj, reference_ft5_base_adj),
    'flan_t5_base_KeyBert_adj': (Flan_T5_base_KeyBert_predictions_adj, reference_ft5_base_adj),
    'flan_t5_large': (Flan_t5_large_baseline_predictions, reference_ft5_large),
    'flan_t5_large_Bio': (Flan_t5_large_Bio_predictions, reference_ft5_large),
    'flan_t5_large_Clinical': (Flan_t5_large_Clinical_predictions, reference_ft5_large),
    'flan_t5_large_KeyBert': (Flan_T5_large_KeyBert_predictions, reference_ft5_large),
    'LED_large_shortdialogue': (LED_LORA_large_shortdialogue_predictions, reference_LED_LORA_large_baseline_shortdialogue),
    'LED_large_Bio_shortdialogues': (LED_LORA_large_shortdialogue_Bio_predictions, reference_LED_LORA_large_baseline_shortdialogue),
    'LED_large_Clinical_shortdialogue': (LED_LORA_large_shortdialogue_Clinical_predictions, reference_LED_LORA_large_baseline_shortdialogue),
    'LED_large_keybert_shortdialogues': (LED_LORA_large_shortdialogue_keybert_predictions, reference_LED_LORA_large_baseline_shortdialogue),
}

# Calculate scores for each model
scores = {}
for model_name, (predictions, references) in models_predictions_references.items():
    scores[model_name] = calculate_rouge(predictions, references)

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(scores, orient='index')

# Reset the index to have a column for the model names
df.reset_index(inplace=True)
df.rename(columns={'index': 'model'}, inplace=True)

# Insert empty rows at the specified positions
df.loc[3.4] = [''] * len(df.columns)  # Insert before 'flan_t5_base_KeyBert'
df.loc[7.4] = [''] * len(df.columns)  # Insert before 'flan_t5_base_KeyBert_adj'
df.loc[11.4] = [''] * len(df.columns)  # Insert before 'flan_t5_large_KeyBert'

# Sort the index to position the empty rows correctly
df.sort_index(inplace=True)

# Display the DataFrame
df


,model,rouge1_recall,rouge1_precision,rouge1_f1,rouge2_recall,rouge2_precision,rouge2_f1,rougeL_recall,rougeL_precision,rougeL_f1
0.0,flan_t5_base,0.504926,0.260104,0.289646,0.238523,0.1016,0.121821,0.432544,0.230183,0.25288
1.0,flan_t5_base_Bio,0.515908,0.260468,0.291812,0.242375,0.10598,0.125107,0.4371,0.229046,0.25268
2.0,flan_t5_base_Clinical,0.508469,0.246695,0.28176,0.23414,0.10187,0.11998,0.426827,0.216081,0.242329
3.0,flan_t5_base_KeyBert,0.516668,0.24998,0.282631,0.249743,0.103053,0.123963,0.442613,0.220257,0.246436
3.4,,,,,,,,,,
4.0,flan_t5_base_adj,0.53867,0.257973,0.297801,0.257715,0.098484,0.122007,0.463502,0.231401,0.262461
5.0,flan_t5_base_Bio_adj,0.522582,0.2518,0.285524,0.243753,0.099906,0.119974,0.444374,0.220876,0.247951
6.0,flan_t5_base_Clinical_adj,0.529982,0.246754,0.289024,0.246005,0.099019,0.119002,0.447922,0.216439,0.249797
7.0,flan_t5_base_KeyBert_adj,0.53134,0.245977,0.286838,0.257137,0.104204,0.127201,0.455192,0.216191,0.249707
7.4,,,,,,,,,,


In [77]:
from rouge_score import rouge_scorer
import pandas as pd

# Define a function to calculate ROUGE scores
def calculate_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1_recall = []
    rouge1_precision = []
    rouge1_f1 = []
    rouge2_recall = []
    rouge2_precision = []
    rouge2_f1 = []
    rougeL_recall = []
    rougeL_precision = []
    rougeL_f1 = []

    for pred, ref in zip(predictions, references):
        score = scorer.score(pred, ref)
        rouge1_recall.append(score['rouge1'].recall)
        rouge1_precision.append(score['rouge1'].precision)
        rouge1_f1.append(score['rouge1'].fmeasure)
        rouge2_recall.append(score['rouge2'].recall)
        rouge2_precision.append(score['rouge2'].precision)
        rouge2_f1.append(score['rouge2'].fmeasure)
        rougeL_recall.append(score['rougeL'].recall)
        rougeL_precision.append(score['rougeL'].precision)
        rougeL_f1.append(score['rougeL'].fmeasure)

    avg_scores = {
        'rouge1_recall': sum(rouge1_recall) / len(rouge1_recall),
        'rouge1_precision': sum(rouge1_precision) / len(rouge1_precision),
        'rouge1_f1': sum(rouge1_f1) / len(rouge1_f1),
        'rouge2_recall': sum(rouge2_recall) / len(rouge2_recall),
        'rouge2_precision': sum(rouge2_precision) / len(rouge2_precision),
        'rouge2_f1': sum(rouge2_f1) / len(rouge2_f1),
        'rougeL_recall': sum(rougeL_recall) / len(rougeL_recall),
        'rougeL_precision': sum(rougeL_precision) / len(rougeL_precision),
        'rougeL_f1': sum(rougeL_f1) / len(rougeL_f1),
    }
    return avg_scores

# Define your predictions and references for each model
models_predictions_references = {
    'flan_t5_base': (Flan_t5_base_baseline_predictions, reference_ft5_base),
    'flan_t5_base_Bio': (Flan_t5_base_Bio_predictions, reference_ft5_base),
    'flan_t5_base_Clinical': (Flan_t5_base_Clinical_predictions, reference_ft5_base),
    'flan_t5_base_KeyBert': (Flan_T5_base_KeyBert_predictions, reference_ft5_base),
    'flan_t5_base_adj': (Flan_t5_base_baseline_predictions_adj, reference_ft5_base_adj),
    'flan_t5_base_Bio_adj': (Flan_t5_base_Bio_predictions_adj, reference_ft5_base_adj),
    'flan_t5_base_Clinical_adj': (Flan_t5_base_Clinical_predictions_adj, reference_ft5_base_adj),
    'flan_t5_base_KeyBert_adj': (Flan_T5_base_KeyBert_predictions_adj, reference_ft5_base_adj),
    'flan_t5_large': (Flan_t5_large_baseline_predictions, reference_ft5_large),
    'flan_t5_large_Bio': (Flan_t5_large_Bio_predictions, reference_ft5_large),
    'flan_t5_large_Clinical': (Flan_t5_large_Clinical_predictions, reference_ft5_large),
    'flan_t5_large_KeyBert': (Flan_T5_large_KeyBert_predictions, reference_ft5_large),
    'LED_large_shortdialogue': (LED_LORA_large_shortdialogue_predictions, reference_LED_LORA_large_baseline_shortdialogue),
    'LED_large_Bio_shortdialogues': (LED_LORA_large_shortdialogue_Bio_predictions, reference_LED_LORA_large_baseline_shortdialogue),
    'LED_large_Clinical_shortdialogue': (LED_LORA_large_shortdialogue_Clinical_predictions, reference_LED_LORA_large_baseline_shortdialogue),
    'LED_large_keybert_shortdialogues': (LED_LORA_large_shortdialogue_keybert_predictions, reference_LED_LORA_large_baseline_shortdialogue),
}

# Calculate scores for each model
scores = {}
for model_name, (predictions, references) in models_predictions_references.items():
    scores[model_name] = calculate_rouge(predictions, references)

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(scores, orient='index')

# Reset the index to have a column for the model names
df.reset_index(inplace=True)
df.rename(columns={'index': 'model'}, inplace=True)

# Insert empty rows at the specified positions
df.loc[3.4] = [''] * len(df.columns)  # Insert before 'flan_t5_base_KeyBert'
df.loc[7.4] = [''] * len(df.columns)  # Insert before 'flan_t5_base_KeyBert_adj'
df.loc[11.4] = [''] * len(df.columns)  # Insert before 'flan_t5_large_KeyBert'

# Sort the index to position the empty rows correctly
df.sort_index(inplace=True)

# Format the DataFrame to replace NaN with empty strings
df = df.fillna('')

# Display the DataFrame without the index
print(df.to_string(index=False))


                           model rouge1_recall rouge1_precision rouge1_f1 rouge2_recall rouge2_precision rouge2_f1 rougeL_recall rougeL_precision rougeL_f1
                    flan_t5_base      0.504926         0.260104  0.289646      0.238523           0.1016  0.121821      0.432544         0.230183   0.25288
                flan_t5_base_Bio      0.515908         0.260468  0.291812      0.242375          0.10598  0.125107        0.4371         0.229046   0.25268
           flan_t5_base_Clinical      0.508469         0.246695   0.28176       0.23414          0.10187   0.11998      0.426827         0.216081  0.242329
            flan_t5_base_KeyBert      0.516668          0.24998  0.282631      0.249743         0.103053  0.123963      0.442613         0.220257  0.246436
                                                                                                                                                           
                flan_t5_base_adj       0.53867         0.257973 

**flan_t5 Models:**

- flan_t5_large and its variants (Bio, Clinical, KeyBert) generally outperform the base versions in all ROUGE metrics.
- The flan_t5_large model has the highest ROUGE-1 F1 score (0.3269), indicating a better balance between precision and recall in capturing relevant information compared to other models.

**LED Models:**

- LED models, including LED_large_shortdialogue and its variants, exhibit high precision scores (e.g., ROUGE-1 precision of 0.6198 for LED_large_shortdialogue) but much lower recall scores.
- This indicates that while the LED models are good at generating concise summaries that contain relevant information, they miss a lot of the reference content, reflected in their lower recall scores.

**Balanced Performance:**

- The flan_t5_large models show a more balanced performance across recall, precision, and F1 scores, making them more reliable for generating comprehensive summaries.
- The variants like flan_t5_large_Bio and flan_t5_large_Clinical also demonstrate strong performance, indicating the effectiveness of the model's adaptation to specific domains.

**Trade-offs:**

The high precision but low recall of LED models suggest they are better suited for tasks where the emphasis is on precision over recall. Conversely, the flan_t5 models' balanced performance makes them suitable for tasks requiring a good balance of both.

**Conclusion:**

The flan_t5 models, particularly the flan_t5_large and its domain-specific variants, provide the best overall performance in terms of ROUGE scores, indicating their effectiveness in generating high-quality summaries. The LED models, while precise, may need further tuning to improve recall and overall summary completeness.